DATA CLEANING
 - REMOVING UNNECESSARY COLUMNS
 - RENAMING COLUMNS
 - CONVERTING DATE OBJECT INTO DATE TYPE
 - DROPING EXTRA RAWS
 - REMOVING NULL ROWS
 - EDITING DATE AND HOURS ENTRY 

In [22]:
import pandas as pd

df = pd.read_csv('../data/raw/vehicle-volume-data-feed-prophet-model.csv')

Removing unnecessary columns
Drop the columns that are unnecessary by prophet model

Columns:
 - holiday               
 - temp                 
 - rain_1h              
 - snow_1h              
 - clouds_all             
 - weather_main          
 - weather_description

In [23]:
df.drop(columns=['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description'], inplace=True)

Renaming columns
 - date_time -> ds (timestamp): This will serve as x-axis
 - traffic_volume -> y (vehicle_count): This will serve as y-axis

In [24]:
df.rename(columns={'date_time': 'ds', 'traffic_volume': 'y'}, inplace=True)

Converting date object into date type

Prophet model cannot handle date object so it must be converted into date type

In [25]:
df['ds'] = pd.to_datetime(df['ds'], format="%d-%m-%Y %H:%M")

Data loading

Droping extra rows

Prophet model can become highly accurate when making a Traffic Time Series Forecasting when fed with 1yr set of data.
Our raw data, specifically in ds (date_time) includes hour, so 1 day data must have 24 rows and a year has 365 days.

24*365 = 8760 rows (must be near to this number)
but this raw data has 48k rows, so we need to drop 48203 - 8760 = 39443 rows

11134 <-> 20344

In [26]:
# Drop rows from 0 to 11133 and 20344 to 48203
df.drop(index=list(range(0, 11132)) + list(range(20344, 48204)), inplace=True)

# reset index numbering
df = df.reset_index(drop=True)

Removing rows with null value

In [27]:
(df[df.isnull().any(axis=1)]).count() # from any column, count the number of rows with null value

y     0
ds    0
dtype: int64

In [28]:
df.head()
#df.info()
df.count() # 48k raws before processing

y     9212
ds    9212
dtype: int64

Check for duplicate entries

In [29]:
df['ds'].unique() 
#8094 unique values but the total rows are 9212 which means
#1118 rows are duplicated

<DatetimeArray>
['2014-01-01 00:00:00', '2014-01-01 01:00:00', '2014-01-01 02:00:00',
 '2014-01-01 03:00:00', '2014-01-01 04:00:00', '2014-01-01 05:00:00',
 '2014-01-01 06:00:00', '2014-01-01 07:00:00', '2014-01-01 08:00:00',
 '2014-01-01 09:00:00',
 ...
 '2015-12-31 08:00:00', '2015-12-31 09:00:00', '2015-12-31 10:00:00',
 '2015-12-31 12:00:00', '2015-12-31 13:00:00', '2015-12-31 15:00:00',
 '2015-12-31 16:00:00', '2015-12-31 19:00:00', '2015-12-31 21:00:00',
 '2015-12-31 23:00:00']
Length: 8094, dtype: datetime64[ns]

Aggregate Duplicate Values

In [30]:
y_value = df['y']
df = df.groupby('ds', as_index=False)['y'].sum()

In [31]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8094 entries, 0 to 8093
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      8094 non-null   datetime64[ns]
 1   y       8094 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 126.6 KB


Edit Date and Hours Entry

In [32]:
full_range = pd.date_range(start='2024-01-01 00:00', end='2024-12-31 23:00', freq='H')
df = df.set_index('ds').reindex(full_range).fillna(0).rename_axis('ds').reset_index()

C:\Users\imper\AppData\Local\Temp\ipykernel_16168\3565427946.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(start='2024-01-01 00:00', end='2024-12-31 23:00', freq='H')


In [33]:
df_len = len(df)
y_value = y_value.drop(index=range(df_len, len(y_value)))

In [ ]:
df['y'] = y_value # pass the value

In [38]:
df.loc[df.duplicated() == True, 'ds'] # check for duplicate entry
(df.loc[df.duplicated() == True, 'ds']).count() # count = 0 no duplicates

np.int64(0)

Save as CSV file

In [39]:
df.to_csv('../data/processed/vehicle-data-feed-prophet-model.csv', index=False)